### Training


In [18]:
%load_ext tensorboard

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import datetime
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import librosa
import math
import random


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [19]:
DATA_PATH = "data.json"

VAL_SIZE = 0.2 # percentage of dataset
TEST_SIZE = 0.1 # percentage of dataset


In [ ]:
# load training data from json file

with open(DATA_PATH, "r") as f:
    data = json.load(f)

X = np.array(data["mfcc"])
y = np.array(data["labels"])


In [ ]:
# split data into train, validation and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=VAL_SIZE)

# add an axis to input sets to match the shape CNN expects (last axis is like channel in color images)
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]


In [20]:
# choose hyperparameters to tune

hparams_log_dir = 'logs/hparam_tuning' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([32, 64, 96]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.4))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(hparams_log_dir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )


In [21]:
def train_test_model(hparams):
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(keras.layers.Dropout(hparams[HP_DROPOUT]))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer=hparams[HP_OPTIMIZER],
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    fit_log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=hparams_log_dir, histogram_freq=1)

    hparams_callback = hp.KerasCallback(hparams_log_dir, hparams)

    model.fit(x=X_train,
              y=y_train,
              validation_data=(X_validation, y_validation),
              batch_size=32,
              epochs=30,
              callbacks=[tensorboard_callback, hparams_callback])


def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    train_test_model(hparams)

In [22]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/30
212/212 [==============================] - 8s 26ms/step - loss: 1.6700 - accuracy: 0.4360 - val_loss: 1.6408 - val_accuracy: 0.4374
Epoch 2/30
212/212 [==============================] - 4s 20ms/step - loss: 0.9748 - accuracy: 0.6667 - val_loss: 1.6360 - val_accuracy: 0.5260
Epoch 3/30
212/212 [==============================] - 4s 20ms/step - loss: 0.7560 - accuracy: 0.7420 - val_loss: 1.1123 - val_accuracy: 0.6478
Epoch 4/30
212/212 [==============================] - 3s 16ms/step - loss: 0.6074 - accuracy: 0.7951 - val_loss: 0.9052 - val_accuracy: 0.7204
Epoch 5/30
212/212 [==============================] - 3s 16ms/step - loss: 0.5077 - accuracy: 0.8240 - val_loss: 0.9389 - val_accuracy: 0.6980
Epoch 6/30
212/212 [==============================] - 3s 16ms/step - loss: 0.4269 - accuracy: 0.8528 - val_loss: 1.0337 - val_accuracy: 0.6974
Epoch 7/30
212/212 [==============================] - 3s 16ms

In [ ]:
%tensorboard --logdir logs/hparam_tuning

In [ ]:
%tensorboard --logdir logs/fit